In [1]:
# for images
import os, cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from processor.pre_processor import DatasetPreprocessor

# for transfer learning
from processor.data_loader import DataLoader
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [2]:
data_dir = "data"
dataset_dir = os.path.join(data_dir, "dataset")
dataset_2_dir = os.path.join(data_dir, "dataset_2")
output_dir = os.path.join(data_dir, "augmented_images")
labels_path = os.path.join(data_dir, "labels", "names.json")
partition_path = os.path.join(data_dir, "partition")
train_dir = os.path.join(partition_path, 'train')
val_dir = os.path.join(partition_path, 'val')
train_classes = sorted(os.listdir(train_dir))
val_classes = sorted(os.listdir(val_dir))

In [5]:
# Set the CUDA device order (optional)
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU'), 'GPU')

# Enable GPU growth
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
# Load the MobileNetV2 model without the top layers and with ImageNet weights
img_rows, img_cols = 224, 224
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))

for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Descirbe the architecture of the MobileNetV2
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [9]:
preprocessor = DatasetPreprocessor(dataset_dir)
data_loader = DataLoader(dataset_dir, labels_path)

In [12]:
# run only if there are images to be renamed
preprocessor.rename_images()
preprocessor.augment_dataset(output_dir)
preprocessor.generate_labels(labels_path)

In [29]:
batch_size = 32
data_generator = ImageDataGenerator(rescale=1./255)
custom_classes = os.listdir(train_dir)

In [30]:
# Create a training data generator
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    classes=custom_classes
)

# Create a validation data generator
validation_generator = data_generator.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    classes=custom_classes
)

Found 10960 images belonging to 23 classes.
Found 2756 images belonging to 23 classes.
